In [1]:
# ---SECTION 1 ---
# Libraries and data loading
from  sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np

bc= load_breast_cancer()
train_x, train_y= bc.data[:400], bc.target[:400]
test_x, test_y= bc.data[400:], bc.target[400:]

In [2]:
---SECTION 2 ---
# Create the ensemble's base learners and meta-learner
# Append base learners to a list for ease of access
base_learners= []
knn= KNeighborsClassifier(n_neighbors=2)
base_learners.append(knn)
dtr= DecisionTreeClassifier(max_depth=4, random_state=123456)
base_learners.append(dtr)
mlpc= MLPClassifier(hidden_layer_sizes=(100, ),
solver='lbfgs', random_state=123456)
base_learners.append(mlpc)
meta_learner= LogisticRegression(solver='lbfgs')

SyntaxError: invalid syntax (2501587762.py, line 1)

In [ ]:
# ---SECTION 3 ---
# Create the training metadata
# Create variables to store metadata and their targets
meta_data= np.zeros((len(base_learners), len(train_x)))
meta_targets= np.zeros(len(train_x))
# Create the cross-validation folds
KF= KFold(n_splits=5)
meta_index= 0
for train_indices, test_indices in KF.split(train_x):
# Train each learner on the K-1 folds and create
# metadata for the Kth fold
    for i in range(len(base_learners)):
        learner= base_learners[i]
        learner.fit(train_x[train_indices], train_y[train_indices])
        predictions= learner.predict_proba(train_x[test_indices])[:,0]
        meta_data[i][meta_index:meta_index+len(test_indices)] = predictions

    meta_targets[meta_index:meta_index+len(test_indices)] = train_y[test_indices]
    meta_index+= len(test_indices)
# Transpose the metadata to be fed into the meta-learner
meta_data= meta_data.transpose()

In [ ]:
# ---SECTION 4 ---
# Create the metadata for the test set and evaluate the base learners
test_meta_data= np.zeros((len(base_learners), len(test_x)))
base_acc= []
for i in range(len(base_learners)):
    learner= base_learners[i]
    learner.fit(train_x, train_y)
    predictions= learner.predict_proba(test_x)[:,0]
    test_meta_data[i] = predictions

    acc= metrics.accuracy_score(test_y, learner.predict(test_x))
    base_acc.append(acc)
    
test_meta_data= test_meta_data.transpose()

In [ ]:
# ---SECTION 5 ---
# Fit the meta-learner on the train set and evaluate it on the test set
meta_learner.fit(meta_data, meta_targets)
ensemble_predictions= meta_learner.predict(test_meta_data)
acc= metrics.accuracy_score(test_y, ensemble_predictions)
# ---SECTION 6 ---
# Print the results
print('Acc Name')
print('-'*20)
for i in range(len(base_learners)):
    learner= base_learners[i]
    print(f'{base_acc[i]:.2f}{learner.__class__.__name__}')
print(f'{acc:.2f}Ensemble')